In [22]:
from getdist import loadMCSamples
from scipy.stats import gaussian_kde
from sklearn.neighbors import KernelDensity
import numpy as np


#### Bayes factors for LCDM+$\Omega_K$ vs LCDM using Planck+DESI data. 
For nested models the Savage-Dickey density ratio provides a way to calculate the Bayesian evidence (or log Bayes-factors). It is given by

$$\text{BF}_{01} = \frac{p(\theta_0 | \text{data}, \mathcal{M}_1)}{p(\theta_0 | \mathcal{M}_1)}$$

where:
- $\text{BF}_{01}$ is the Bayes factor comparing model $\mathcal{M}_0$ (LCDM, $\Omega_K = 0$) to model $\mathcal{M}_1$ (LCDM+$\Omega_K$)
- $p(\theta_0 | \text{data}, \mathcal{M}_1)$ is the posterior density of parameter $\theta$ at the null value $\theta_0$ under the extended model
- $p(\theta_0 | \mathcal{M}_1)$ is the prior density at the null value under the extended model
- For our case: $\theta = \Omega_K$ and $\theta_0 = 0$

In [23]:
samples = loadMCSamples("./cosmo_input/chains/Planck_DESI_LCDM_MCMC_Omk")
omega_k_param = 'omk'
param_names = samples.getParamNames().list()
param_index = param_names.index(omega_k_param)
omega_k_samples = samples.samples[:, param_index]
param_ranges = samples.ranges    
# Get bounds using getLower and getUpper methods
lower = param_ranges.getLower(omega_k_param)
upper = param_ranges.getUpper(omega_k_param)
print(lower, upper)

-0.005 0.01


In [24]:
# Scipy.stats method
kde = gaussian_kde(omega_k_samples,bw_method='scott')
#check posterior is normalised
normalization_factor = kde.integrate_box_1d(lower, upper)
print(f"Normalization Factor: {normalization_factor:.4f}")

# Evaluate the posterior density at Omega_k = 0
posterior_at_0 = kde.evaluate(0)[0]
print(f"Estimated Posterior Density at Ωk=0: {posterior_at_0:.4f}")
prior_at_0 = 1 / (upper - lower)
savage_dickey_ratio = posterior_at_0 / prior_at_0
print(f"Savage-Dickey Ratio at Ωk=0: {savage_dickey_ratio:.4f}")
print(f"Log Bayes Factor at Ωk=0: {np.log(savage_dickey_ratio):.4f}")

Normalization Factor: 1.0000
Estimated Posterior Density at Ωk=0: 97.0856
Savage-Dickey Ratio at Ωk=0: 1.4563
Log Bayes Factor at Ωk=0: 0.3759


In [35]:
#Sklearn method
kde = KernelDensity(kernel='gaussian', bandwidth='scott')
kde.fit(omega_k_samples[:, None])
# get normalisation
grid = np.linspace(lower, upper, 1000)[:, None]
norm = np.trapz(np.exp(kde.score_samples(grid)), x=grid[:, 0])
print(norm)
posterior_at_0 = np.exp(kde.score_samples(np.zeros((1,1))))[0] / norm
print(f"Estimated Posterior Density at Ωk=0: {posterior_at_0:.4f}")
savage_dickey_ratio = posterior_at_0 / prior_at_0
print(f"Savage-Dickey Ratio at Ωk=0: {savage_dickey_ratio:.4f}")
print(f"Log Bayes Factor at Ωk=0: {np.log(savage_dickey_ratio):.4f}")

0.0727982796284852
Estimated Posterior Density at Ωk=0: 66.7355
Savage-Dickey Ratio at Ωk=0: 1.0010
Log Bayes Factor at Ωk=0: 0.0010
